# Accessing Truth Information

An important aspect at this stage in the design of EIC experiments is the comparison of the so-called 'truth' information with simulated variables. In this notebook we will go over how to access the truth information in an ePIC event.

## Importing uproot

Depending on the versions of uproot and XRootD that you have installed, you may encouter a warning from uproot below. Nevertheless, because of the simple data format of the ATHENA ROOT files, we are able to ignore this warning.

In [ ]:
import uproot as ur
import os
print('Uproot version: ' + ur.__version__)

## Opening a file with uproot

To test uproot, we will open a sample file (a single-particle simulation of interest to those who wish to study detector performance):

In [ ]:
file_name = 'pythia8NCDIS_18x275_minQ2=10_beamEffects_xAngle=-0.025_hiDiv_1.0000.eicrecon.tree.edm4eic.root'
file_path = os.path.join('..', 'data', file_name)
file_path

In [ ]:
events = ur.open(file_path + ':events')

## Truth information in the `MCParticles` branch

The truth information is stored in the `MCParticles` branch. This includes the true *generated particles* in the simulation, subject to certain conditions. For the purpose of end-user analysis of centrally produced simulation files, the conditions are essentially that only primary particles are included.

Several fields are available for the truth information:

In [ ]:
events['MCParticles'].keys()

Besides the particle data group code `PDG`, the indices of the parent and daughter tracks, and the generator status `generatorStatus`, you will also see the creation vertex position (`vertex`) and momentum (`momentum`), as well as the endpoint position (`endpoint`) and momentum (`momentumAtEndpoint`) of the particle. Thus, `momentum.x` corresponds to the `x` component of the momentum at the creation vertex. Let's retrieve these momenta, as well as the `PDG` code and `generatorStatus`.

In [ ]:
PDG = events['MCParticles.PDG'].array()
generatorStatus = events['MCParticles.generatorStatus'].array()
psx,psy,psz = events['MCParticles.momentum.x'].array(), events['MCParticles.momentum.y'].array(), events['MCParticles.momentum.z'].array()

As expected, for this file the `PDG` code corresponds to many different particles, e.g. for one event we find:

In [ ]:
PDG[100]

Let's compare this with the generator status, and keep in mind the following values in the HepMC3 standard:
- 0: undefined and should not occur for well-formed input,
- 1: macroscopically stable particles that are thrown into the detector simulation,
- 2: unstable particles which have been decayed by the event generator,
- 4: incoming beam particles,
- &gt; 10: reserved for event generator use, e.g. virtual photons in hard scattering.

In [ ]:
generatorStatus[100]

We can also look at the total momentum. For this, we are importing the `numpy` library to use the `sqrt` function on arrays.

In [ ]:
import numpy as np
p = np.sqrt(psx**2 + psy**2 + psz**2)

In [ ]:
p[100]

## Making a simple plot

We can now create a simple plot of the angular (theta) distribution of the generated particles.

In [ ]:
import awkward as ak
theta = np.arctan2(np.sqrt(psx**2 + psy**2), psz)
theta[generatorStatus == 1]

In [ ]:
import matplotlib.pyplot as plt
plt.hist(-ak.flatten(theta[generatorStatus == 1]), range = (-np.pi, 0), bins = 50)
plt.xlabel('Initial Scattering Angle, $-\\theta$ [rad]')
plt.ylabel('Number of events')
plt.show()

# Exercise

Create a stacked histogram that separates the theta distributions by particle type. Think (or determine) which particle types will be most relevant here.

In [ ]:
# your code here



### Possible solution

In [ ]:
stacked_theta = [
    -ak.flatten(theta[np.logical_and(PDG == 11, generatorStatus == 1)]),
    -ak.flatten(theta[np.logical_and(PDG == 22, generatorStatus == 1)]),
    -ak.flatten(theta[np.logical_and(np.logical_or(PDG == 211, PDG == -211), generatorStatus == 1)]),
]
plt.hist(stacked_theta, range = (-np.pi, 0), bins = 50, stacked = True, label = ["e", "$\\gamma$", "$\\pi^\\pm$"])
plt.xlabel('Initial Scattering Angle, $-\\theta$ [rad]')
plt.ylabel('Number of events')
plt.legend()
plt.show()